In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime
from os import environ
from services.zayev.service_layer.agent import PPOAgent as ZPPO
from matplotlib.dates import relativedelta
import numpy as np


2023-10-12 08:47:29.882113: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<Figure size 1800x900 with 0 Axes>

In [2]:

starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

db_params = {
            'database': environ.get("POSTGRES_DB"),
            'user': environ.get("POSTGRES_USER"),
            'password': environ.get("POSTGRES_PASSWORD"),
            'host': environ.get("DB_HOST"),
            'port': environ.get("DB_PORT"),
        }
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 150, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "test_steps": 10
} 



zayev = ZPPO(env_config)

/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
print(zayev.Actor.Actor.summary())

In [3]:
zayev.save()

2023-10-12 08:07:19.003279: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-10-12 08:07:19.027738: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_7/bias/Assign' id:331 op device:{requested: '', assigned: ''} def:{{{node dense_7/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_7/bias, dense_7/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [27]:
np.random.seed(np.random.randint(0,10))
# zayev.run_batch()
zayev.run_multiprocesses(num_worker=4)

episode: 1/20, score: 39906.2987531269, average: 51298.79 
episode: 2/20, score: 49534.39129205205, average: 51443.67 
episode: 3/20, score: 19233.492472887578, average: 51711.74 
episode: 4/20, score: 48270.40070902475, average: 50924.84 
episode: 5/20, score: 69217.38230239284, average: 50976.45 
episode: 6/20, score: 30946.028292381467, average: 49982.27 
episode: 7/20, score: 28649.565926081166, average: 49557.31 
episode: 8/20, score: 51980.37861912092, average: 49358.87 
episode: 9/20, score: 28345.257949919425, average: 48917.20 
episode: 10/20, score: 39548.05308319027, average: 48929.30 
episode: 11/20, score: 63914.50972054481, average: 49107.09 
episode: 12/20, score: 42290.3127935573, average: 48914.28 
episode: 13/20, score: 5829.986113611289, average: 47805.75 
episode: 14/20, score: 87615.32218963967, average: 48412.80 
episode: 15/20, score: 66637.21218295318, average: 48176.11 
episode: 16/20, score: 80654.65249981834, average: 48760.39 
episode: 17/20, score: 49897.90

In [3]:
# print(zayev.Actor.Actor)
# [-1.         -0.99999756  0.9822062  -1.         -0.99031776]
# zayev.load()
zayev.test()

2023-10-12 08:47:47.496628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-10-12 08:47:47.524792: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_7/bias/Assign' id:331 op device:{requested: '', assigned: ''} def:{{{node dense_7/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_7/bias, dense_7/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-10-12 08:47:


stepping_a [-0.08306313 -0.46334487 -0.45359194  0.3293842   0.33547658  0.33513927]
[[1.54616108e+02]
 [3.28705574e+00]
 [4.15571983e+02]
 [7.90097345e+03]
 [2.72435697e+04]
 [3.69284949e+03]]
[[  86.68822578 1750.87558874  802.25604172   86.68913453  849.91507752
    86.69170479   86.8         230.41474654]
 [  24.90723658 5246.00036379 2281.14327628   24.90904755 2478.84908815
    24.91665431   25.          800.        ]
 [ 156.65782667 1050.09201815  524.57603361  156.6650944   592.58090151
   156.68299037  158.33333333  126.31578947]
 [  37.72441822  600.42571078  306.14990314   37.73233772  292.60462715
    37.73751442   37.5         533.33333333]
 [  21.2407929  1751.68870569  933.78543534   21.24463182  916.00216194
    21.25213886   21.33333333  937.5       ]]
[0]
change_in_shares for index: 0 -95.5502177799411 937.5 802.2560417206539
change_in_shares for index: 1 -1854.9631194263634 937.5 2281.1432762782492
change_in_shares for index: 2 -288.72283067459233 937.5 524.57603360

In [5]:
print(zayev.previous_state)
print(zayev.previous_actn)
print(zayev.latest_state)

[array([[[8.89013229e+01, 1.82192255e+03, 8.64895951e+02, 8.89080660e+01,
         7.91909898e+02, 8.89155285e+01, 8.85273127e+01, 4.74517534e+02],
        [2.42320190e+01, 5.37665102e+03, 2.47012221e+03, 2.42377512e+01,
         2.49610978e+03, 2.42519959e+01, 2.43558116e+01, 1.00000000e+00],
        [1.20238241e+02, 1.10505589e+03, 5.62963442e+02, 1.20281065e+02,
         5.49843722e+02, 1.20516134e+02, 1.30199188e+02, 1.00000000e+00],
        [4.08102075e+01, 6.45030271e+02, 3.33120493e+02, 4.08154987e+01,
         3.26101390e+02, 4.08412789e+01, 4.03972509e+01, 5.06243777e+02],
        [2.11843179e+01, 1.86668363e+03, 9.24151624e+02, 2.11867193e+01,
         8.34191489e+02, 2.11943689e+01, 2.12600581e+01, 1.85350216e+03]]]), array([[[1.72134146e+02],
        [3.53908166e+00],
        [4.54222948e+02],
        [7.23837447e+03],
        [2.83403424e+04],
        [4.14611601e+03]]]), array([[0]])]
[ 0.0979326  -0.50656664 -0.49343336  0.29890707  0.30164826  0.30151203]
[array([[[8.84

In [6]:
ztt = zayev.latest_state
stt_stt = zayev.latest_state[0][0]
print(stt_stt)

total_potfolio = zayev.latest_state[2][0]
for st_dta in stt_stt:
    total_potfolio = total_potfolio + st_dta[0]*st_dta[7]
print(total_potfolio)

[[8.84027568e+01 1.83942439e+03 8.83446877e+02 8.84072221e+01
  7.93678703e+02 8.84156890e+01 8.89013229e+01 4.74517534e+02]
 [2.40758430e+01 5.37697078e+03 2.41378245e+03 2.40833662e+01
  2.35209760e+03 2.40955000e+01 2.42320190e+01 1.00000000e+00]
 [1.14772717e+02 1.11613447e+03 5.62077427e+02 1.14829903e+02
  5.58962911e+02 1.14964128e+02 1.20238241e+02 1.00000000e+00]
 [4.14821007e+01 6.50706205e+02 3.31000321e+02 4.15090196e+01
  3.27949541e+02 4.15361948e+01 4.08102075e+01 5.06243777e+02]
 [2.10941760e+01 1.86811972e+03 9.54179757e+02 2.10983064e+01
  8.27411385e+02 2.11071421e+01 2.11843179e+01 1.85350216e+03]]
[102185.66283696]


In [8]:
tm = starting_time_step + relativedelta(hours=25)
print(tm)

1995-01-02 11:00:00


In [6]:
zayev.latest_state

[array([[[  86.98892013, 1780.88718796,  806.78320262,   86.99418474,
           815.32319724,   87.01223627,   86.22798652,  231.66143027],
         [  24.55947431, 5311.76025676, 2294.7676268 ,   24.5622656 ,
          2544.55969405,   24.56600397,   24.61056939,  795.63826776],
         [ 139.2253307 , 1086.44600003,  505.13989644,  139.25715658,
           561.13738873,  139.33339698,  144.20254547,  126.31578947],
         [  38.81905216,  614.39851888,  328.47856092,   38.82553138,
           308.16662239,   38.84516044,   38.48886214,  533.33333333],
         [  21.18280565, 1786.27803173,  976.56194102,   21.18713267,
           841.46566394,   21.19766482,   21.08023049,  937.5       ]]]),
 array([[[1.71481016e+02],
         [3.49922248e+00],
         [4.32156117e+02],
         [7.54711667e+03],
         [2.72181363e+04],
         [3.93169616e+03]]]),
 array([[0]])]

In [8]:
actp = np.array([1,2,3])*1000
print(actp)

[1000 2000 3000]


In [13]:
log_std = .5 * np.ones(5, dtype=np.float32)
std = np.exp(log_std)

print(log_std)
print(std)

[0.5 0.5 0.5 0.5 0.5]
[1.6487212 1.6487212 1.6487212 1.6487212 1.6487212]
